In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
#tf.autograph.set_verbosity(0)


In [2]:
X=np.array([[1,3],[2,7],[3,6],[2,4],[6,5],[7,6]])
Y=np.array([[1],[0],[1],[0],[1],[1]])
print(X.shape, Y.shape)

(6, 2) (6, 1)


In [6]:
"""Normalize Data
Fitting the weights to the data (back-propagation, covered in next week's lectures) will proceed more quickly if the data is normalized.
 This is the same procedure you used in Course 1 where features in the data are each normalized to have a similar range. The procedure below uses a Keras normalization layer.
  It has the following steps:

create a "Normalization Layer". Note, as applied here, this is not a layer in your model.
'adapt' the data. This learns the mean and variance of the data set and saves the values internally.
normalize the data.
It is important to apply normalization to any future data that utilizes the learned model."""


print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")


Temperature Max, Min pre normalization: 7.00, 1.00
Duration    Max, Min pre normalization: 7.00, 3.00
Temperature Max, Min post normalization: 1.58, -1.13
Duration    Max, Min post normalization: 1.36, -1.61


In [14]:
#Tile/copy our data to increase the training set size and reduce the number of training epochs

Xt = np.tile(Xn,(10000,1))          #1000 is no. of repetation in row and 1 is no. of repetation in column
Yt= np.tile(Y,(10000,1))   
print(Xt.shape, Yt.shape)   

(60000, 2) (60000, 1)


In [15]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [16]:
"""Note 1: The tf.keras.Input(shape=(2,)), specifies the expected shape of the input. This allows Tensorflow to size the weights and bias parameters at this point.
 This is useful when exploring Tensorflow models. This statement can be omitted in practice and Tensorflow will size the network parameters when the input data is 
 
 specified in the model.fit statement.
Note 2: Including the sigmoid activation in the final layer is not considered best practice. It would instead be accounted for in the loss which improves numerical stability.
 This will be described in more detail in a later lab.

The model.summary() provides a description of the network:"""

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [17]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

L1 params =  9 , L2 params =  4


In [18]:
"""Let's examine the weights and biases Tensorflow has instantiated. The weights  𝑊
  should be of size (number of features in input, number of units in the layer) while the bias  𝑏
  size should match the number of units in the layer:

In the first layer with 3 units, we expect W to have a size of (2,3) and  𝑏
  should have 3 elements.
In the second layer with 1 unit, we expect W to have a size of (3,1) and  𝑏
  should have 1 element"""

"Let's examine the weights and biases Tensorflow has instantiated. The weights  𝑊\n  should be of size (number of features in input, number of units in the layer) while the bias  𝑏\n  size should match the number of units in the layer:\n\nIn the first layer with 3 units, we expect W to have a size of (2,3) and  𝑏\n  should have 3 elements.\nIn the second layer with 1 unit, we expect W to have a size of (3,1) and  𝑏\n  should have 1 element"

In [19]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.05938005 -0.6377648  -0.72364426]
 [-0.5575681  -0.08160245 -0.14975691]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.3858496 ]
 [-0.07881188]
 [ 1.0169193 ]] 
b2(1,): [0.]


In [20]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)


"""Epochs and batches
In the compile statement above, the number of epochs was set to 10. This specifies that 
the entire data set should be applied during training 10 times. During training, you see output describing the progress of training that looks like this:

Epoch 1/10
6250/6250 [==============================] - 6s 910us/step - loss: 0.1782
The first line, Epoch 1/10, describes which epoch the model is currently running. For efficiency, 
the training data set is broken into 'batches'. The default size of a batch in Tensorflow is 32. There are
 200000 examples in our expanded data set or 6250 batches. The notation on the 2nd line 6250/6250 [==== is describing which batch has been executed."""

Epoch 1/10
1875/1875 [==============================] - 2s 992us/step - loss: 0.3040
Epoch 2/10
1875/1875 [==============================] - 2s 983us/step - loss: 0.0873
Epoch 3/10
1875/1875 [==============================] - 2s 993us/step - loss: 0.0119
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0037
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0014
Epoch 6/10
1875/1875 [==============================] - 2s 985us/step - loss: 5.2598e-04
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.0820e-04
Epoch 8/10
1875/1875 [==============================] - 2s 988us/step - loss: 8.3055e-05
Epoch 9/10
1875/1875 [==============================] - 2s 956us/step - loss: 3.3281e-05
Epoch 10/10
1875/1875 [==============================] - 2s 980us/step - loss: 1.3361e-05


"Epochs and batches\nIn the compile statement above, the number of epochs was set to 10. This specifies that \nthe entire data set should be applied during training 10 times. During training, you see output describing the progress of training that looks like this:\n\nEpoch 1/10\n6250/6250 [==============================] - 6s 910us/step - loss: 0.1782\nThe first line, Epoch 1/10, describes which epoch the model is currently running. For efficiency, \nthe training data set is broken into 'batches'. The default size of a batch in Tensorflow is 32. There are\n 200000 examples in our expanded data set or 6250 batches. The notation on the 2nd line 6250/6250 [==== is describing which batch has been executed."

In [21]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[  3.0724306 -21.91437     3.723444 ]
 [  9.87312    -2.464961   -2.2128153]] 
b1: [15.111537  -8.899456   6.3772655]
W2:
 [[-22.683119]
 [-23.287506]
 [ 19.798985]] 
b2: [15.174013]


In [22]:
"""Next, we will load some saved weights from a previous training run. This is so that this notebook remains robust to changes in
 Tensorflow over time. Different training runs can produce somewhat different results and the discussion below applies to a particular solution. Feel free to re-run the
 notebook with this cell commented out to see the difference."""

W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])
model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])

In [23]:
"""Once you have a trained model, you can then use it to make predictions. Recall that the output of our model is a probability. 
In this case, the probability of a good roast. To make a decision, one must apply the probability to a threshold. In this case, we will use 0.5

Let's start by creating input data. The model is expecting one or more examples where examples are in the rows of matrix. 
In this case, we have two features so the matrix will be (m,2) where m is the number of examples. Recall, we have normalized 
the input features so we must normalize our test data as well.
To make a prediction, you apply the predict method."""


X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 60ms/step
predictions = 
 [[3.2241868e-08]
 [3.2241868e-08]]


In [25]:
#print(type(predictions))
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

#or
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

<class 'numpy.ndarray'>
decisions = 
[[0.]
 [0.]]
decisions = 
[[0]
 [0]]
